In [2]:
# printing current python version
# and the version of the installed modules
import sys
import pkg_resources
import platform
import subprocess
import os
import sysconfig
import logging
import json
import re


def get_python_version():
    """Get the current Python version."""
    return sys.version.split()[0]

print("Python version:", get_python_version())

Python version: 3.13.2


/var/folders/kz/pzq5xgxj58bb6nqgtmckp_x00000gn/T/ipykernel_13948/3691712826.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [1]:
!pip install ultralytics


  Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
Using cached matplotlib-3.10.3-cp311-cp311-macosx_11_0_arm64.whl (8.1 MB)
Using cached contourpy-1.3.2-cp311-cp311-macosx_11_0_arm64.whl (254 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.

In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.utils import resample, shuffle
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

In [4]:
import os

def rename_images_and_labels(image_dir, label_dir):
    """
    Rename all images in `image_dir` and corresponding label files in `label_dir` to a consistent format `photo<number>`.

    Args:
        image_dir (str): Path to the directory containing image files.
        label_dir (str): Path to the directory containing label files.
    """
    photo_count = 1  # Start numbering from 1

    # Create a list of valid image extensions
    valid_extensions = [".png", ".jpg", ".jpeg"]

    # Process images and labels
    for image_file in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_file)

        # Skip non-files and unsupported formats
        if not os.path.isfile(image_path) or not any(image_file.endswith(ext) for ext in valid_extensions):
            continue

        # Check for a corresponding label file
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_dir, label_file)

        # If the label exists, rename both
        if os.path.exists(label_path):
            new_base_name = f"yayphoto{photo_count}"
            
            # Rename the image
            new_image_name = f"{new_base_name}{os.path.splitext(image_file)[1]}"
            new_image_path = os.path.join(image_dir, new_image_name)
            os.rename(image_path, new_image_path)

            # Rename the label
            new_label_name = f"{new_base_name}.txt"
            new_label_path = os.path.join(label_dir, new_label_name)
            os.rename(label_path, new_label_path)

            print(f"Renamed: {image_file} -> {new_image_name}")
            print(f"Renamed: {label_file} -> {new_label_name}")

            # Increment the counter
            photo_count += 1
        else:
            print(f"No matching label found for image: {image_file}")

if __name__ == "__main__":
    # Specify the paths to the image and label directories
    image_dir = "/Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images"
    label_dir = "/Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/labels"

    rename_images_and_labels(image_dir, label_dir)


Renamed: yayphoto329.jpg -> yayphoto1.jpg
Renamed: yayphoto329.txt -> yayphoto1.txt
Renamed: yayphoto473.jpg -> yayphoto2.jpg
Renamed: yayphoto473.txt -> yayphoto2.txt
Renamed: yayphoto315.jpg -> yayphoto3.jpg
Renamed: yayphoto315.txt -> yayphoto3.txt
Renamed: yayphoto301.jpg -> yayphoto4.jpg
Renamed: yayphoto301.txt -> yayphoto4.txt
Renamed: yayphoto467.jpg -> yayphoto5.jpg
Renamed: yayphoto467.txt -> yayphoto5.txt
Renamed: yayphoto8.jpg -> yayphoto6.jpg
Renamed: yayphoto8.txt -> yayphoto6.txt
Renamed: yayphoto498.jpg -> yayphoto7.jpg
Renamed: yayphoto498.txt -> yayphoto7.txt
Renamed: yayphoto659.jpg -> yayphoto8.jpg
Renamed: yayphoto659.txt -> yayphoto8.txt
Renamed: yayphoto117.jpg -> yayphoto9.jpg
Renamed: yayphoto117.txt -> yayphoto9.txt
Renamed: yayphoto671.jpg -> yayphoto10.jpg
Renamed: yayphoto671.txt -> yayphoto10.txt
Renamed: yayphoto88.jpg -> yayphoto11.jpg
Renamed: yayphoto88.txt -> yayphoto11.txt
Renamed: yayphoto665.jpg -> yayphoto12.jpg
Renamed: yayphoto665.txt -> yayphot

In [5]:
import os
import shutil

# Define folders
source_label_folder = "/Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/labels"
source_image_folder = "/Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images"
target_folder = "/Users/yatharthnehva/Downloads/new_contempt_images"

# Create target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Process each label file
for label_file in os.listdir(source_label_folder):
    if label_file.endswith(".txt"):
        label_path = os.path.join(source_label_folder, label_file)

        # Read the label file
        with open(label_path, "r") as file:
            content = file.read().strip()

        # Extract the label (first number in the file)
        label = content.split()[0]

        if label == "1":  # Check if the label is contempt
            # Construct the corresponding image path
            image_path = label_path.replace("labels", "images").replace(".txt", ".jpg")
            
            if os.path.exists(image_path):
                # Copy the image to the target folder
                shutil.copy(image_path, target_folder)
                print(f"Copied: {image_path}")
            else:
                print(f"Image not found for label file: {label_file}")

print("Processing completed.")


Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto564.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto572.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto638.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto371.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto665.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto128.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto470.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto460.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto138.jpg
Copied: /Users/yatharthnehva/Downloads/Emotion Detection Yolov5 Pytorch/test/images/yayphoto501.jpg


In [6]:
import os

def rename_images_in_place(base_dir):
    for emotion_dir in os.listdir(base_dir):
        emotion_path = os.path.join(base_dir, emotion_dir)
        
        # Check if it's a directory
        if os.path.isdir(emotion_path):
            print(f"Renaming images in folder: {emotion_dir}")
            
            # List all files in the emotion folder
            images = os.listdir(emotion_path)
            for idx, img_file in enumerate(images, start=1):
                # Ensure it's a valid file
                old_path = os.path.join(emotion_path, img_file)
                if not os.path.isfile(old_path) or img_file.startswith('._'):
                    continue
                
                # Generate the new name
                new_name = f"{emotion_dir}FER_{idx}{os.path.splitext(img_file)[1]}"
                new_path = os.path.join(emotion_path, new_name)
                
                # Rename the file
                os.rename(old_path, new_path)
                print(f"Renamed: {img_file} -> {new_name}")

# Example usage
base_dir = "/Users/yatharthnehva/Downloads/disgust"  # Replace with the directory path
rename_images_in_place(base_dir)


In [7]:
import os

# Path to the directory containing the images
directory_path = "/Users/yatharthnehva/Downloads/disgust"

# Extract the emotion from the directory name
emotion = os.path.basename(directory_path)

# Iterate through all files in the directory
for index, filename in enumerate(os.listdir(directory_path), start=1):
    # Full path to the file
    file_path = os.path.join(directory_path, filename)
    
    # Skip if not a file
    if not os.path.isfile(file_path):
        continue

    # Get the file extension
    file_extension = os.path.splitext(filename)[-1].lower()
    
    # Skip if the file is not an image
    if file_extension not in [".jpg", ".jpeg", ".png"]:
        continue

    # New filename
    new_filename = f"FER{emotion}_{index}{file_extension}"
    new_file_path = os.path.join(directory_path, new_filename)
    
    # Rename the file
    os.rename(file_path, new_file_path)
    print(f"Renamed: {filename} → {new_filename}")

print("Renaming complete!")


Renamed: FERdisgust_74.png → FERdisgust_1.png
Renamed: FERdisgust_60.png → FERdisgust_2.png
Renamed: FERdisgust_48.png → FERdisgust_3.png
Renamed: FERdisgust_49.png → FERdisgust_4.png
Renamed: FERdisgust_61.png → FERdisgust_5.png
Renamed: FERdisgust_75.png → FERdisgust_6.png
Renamed: FERdisgust_88.png → FERdisgust_7.png
Renamed: FERdisgust_63.png → FERdisgust_8.png
Renamed: FERdisgust_77.png → FERdisgust_9.png
Renamed: FERdisgust_76.png → FERdisgust_10.png
Renamed: FERdisgust_62.png → FERdisgust_11.png
Renamed: FERdisgust_89.png → FERdisgust_12.png
Renamed: FERdisgust_99.png → FERdisgust_13.png
Renamed: FERdisgust_66.png → FERdisgust_14.png
Renamed: FERdisgust_72.png → FERdisgust_15.png
Renamed: FERdisgust_73.png → FERdisgust_17.png
Renamed: FERdisgust_67.png → FERdisgust_18.png
Renamed: FERdisgust_98.png → FERdisgust_19.png
Renamed: FERdisgust_59.png → FERdisgust_20.png
Renamed: FERdisgust_71.png → FERdisgust_21.png
Renamed: FERdisgust_65.png → FERdisgust_22.png
Renamed: FERdisgust_64

In [8]:
import os
import pandas as pd

# Define the base directory for the FER dataset
base_dir = "/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train" 

# Initialize a list to store the information
data = []

# Iterate through the folders in the base directory
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate through the images in the folder
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            
            # Validate that it is a file and has an appropriate extension
            if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Append the information to the data list
                data.append({
                    "path": image_path,
                    "folder_name": folder_name,
                    "image_name": image_name,
                    "emotion": folder_name  # The folder name represents the emotion
                })

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
output_csv = "fer_train_metadata.csv"
df.to_csv(output_csv, index=False)

print(f"CSV file created successfully: {output_csv}")

CSV file created successfully: fer_train_metadata.csv


In [9]:
import pandas as pd
df = pd.read_csv("fer_train_metadata.csv")
# Specify paths
csv_path = "fer_train_metadata.csv"  # Replace with the actual CSV file path
output_csv_path = "filtered_train_csv.csv"  # Path to save the cleaned CSV

# Load the CSV
df = pd.read_csv(csv_path)

# Filter out rows with '._' in the image_path
filtered_df = df[~df['path'].str.contains(r'\._')]

# Save the filtered DataFrame to a new CSV
filtered_df.to_csv(output_csv_path, index=False)

print(f"Filtered CSV saved to {output_csv_path}")

Filtered CSV saved to filtered_train_csv.csv


In [10]:
# loading the dataset
df = pd.read_csv('filtered_train_csv.csv')

In [11]:
df.head()

,path,folder_name,image_name,emotion
0,/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train...,anger,fer0023473.png,anger
1,/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train...,anger,fer0006441.png,anger
2,/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train...,anger,fer0012170.png,anger
3,/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train...,anger,fer0018331.png,anger
4,/Volumes/Extreme SSD/MLPR_VIDEOS/fer2013/train...,anger,fer0024126.png,anger


In [12]:
# number of entries per class
df['emotion'].value_counts()

emotion
neutral      10308
happiness     7528
surprise      3562
sadness       3514
anger         2466
fear           939
disgust        750
Name: count, dtype: int64

In [13]:
import os
import pandas as pd
from PIL import Image

# Define the mapping for MELD emotion labels to integer values
emotion_mapping = {
    "neutral": 0,
    "happiness": 1,
    "sadness": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5,
    "disgust": 6
}

# Load your dataset
df = pd.read_csv("filtered_train_csv.csv")

# Function to check if the file is a valid image
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()  # Check if it's a valid image
        return True
    except (IOError, SyntaxError):
        return False

# Filter valid rows
valid_rows = []
for _, row in df.iterrows():
    image_path = row['path']
    folder_name = row['folder_name']

    # Check if image path exists, is valid, and folder name is in the mapping
    if os.path.exists(image_path) and is_valid_image(image_path) and folder_name in emotion_mapping:
        row['emotion'] = emotion_mapping[folder_name]
        valid_rows.append(row)

# Create a new DataFrame with valid rows only
filtered_df = pd.DataFrame(valid_rows)

# Save the filtered dataset to a new CSV file
filtered_df.to_csv("mapped_emotions_filtered.csv", index=False)

print("Filtered dataset saved to 'mapped_emotions_filtered.csv'.")


Filtered dataset saved to 'mapped_emotions_filtered.csv'.


In [14]:
df_filtered = pd.read_csv("mapped_emotions_filtered.csv")
df_filtered.head()
df_filtered['emotion'].value_counts()

emotion
0    10308
1     7528
5     3562
2     3514
3     2466
4      939
6      750
Name: count, dtype: int64

In [15]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# define transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(64, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [16]:
class EmotionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

        # Create a mapping for string labels if they exist
        if isinstance(df['emotion'].iloc[0], str):
            self.label_mapping = {label: idx for idx, label in enumerate(df['emotion'].unique())}
            self.df['emotion'] = self.df['emotion'].map(self.label_mapping)
        else:
            self.label_mapping = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['path']
        emotion = self.df.iloc[idx]['emotion']

        # Handle missing files
        if not os.path.exists(img_path):
            print(f"Warning: Missing file {img_path}")
            dummy_image = torch.zeros(3, 64, 64)
            return dummy_image, emotion

        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, emotion
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            dummy_image = torch.zeros(3, 64, 64)
            return dummy_image, emotion


In [17]:
train_dataset = EmotionDataset(train_df, transform=train_transform)
val_dataset = EmotionDataset(val_df, transform=val_test_transform)
test_dataset = EmotionDataset(test_df, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes, input_size=(64, 64)):
        super(EmotionCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=9, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=7, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Dynamically compute the flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, *input_size)
            x = self.conv_block1(dummy_input)
            x = self.conv_block2(x)
            x = self.conv_block3(x)
            x = self.conv_block4(x)
            self.flattened_size = x.view(1, -1).size(1)
        print(f"Computed Flattened Size: {self.flattened_size}")

        self.fc1 = nn.Linear(self.flattened_size, 4096)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [19]:
model = EmotionCNN(num_classes=7, input_size=(64, 64))
dummy_input = torch.randn(32, 3, 64, 64)
output = model(dummy_input)
print(output.shape)  # Should output: torch.Size([32, 7])

Computed Flattened Size: 1024
torch.Size([32, 7])


In [24]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN(num_classes=7).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00115)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


Computed Flattened Size: 1024


In [26]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=50):
    best_acc = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        batch_count = 0

        for i, data in enumerate(train_loader):
            try:
                images, labels = data

                # Skip empty batches
                if images is None or labels is None:
                    continue

                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                batch_count += 1

                if i % 50 == 0:  # Print every 50 batches
                    print(f"Epoch {epoch+1}, Batch {i}, Loss: {loss.item():.4f}")

            except Exception as e:
                print(f"Error in batch {i}: {e}")
                continue

        # Avoid division by zero
        epoch_loss = running_loss / batch_count if batch_count > 0 else 0

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        val_batch_count = 0

        with torch.no_grad():
            for data in val_loader:
                try:
                    images, labels = data

                    # Skip empty batches
                    if images is None or labels is None:
                        continue

                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    val_batch_count += 1

                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                except Exception as e:
                    print(f"Error in validation batch: {e}")
                    continue

        # Avoid division by zero
        val_epoch_loss = val_loss / val_batch_count if val_batch_count > 0 else 0
        val_acc = 100 * correct / total if total > 0 else 0

        if val_acc > best_acc:
            best_acc = val_acc

            # Save the model state dictionary (weights only)
            torch.save(model.state_dict(), "best_model_weights.pth")

            # Save the entire model (architecture + weights)
            torch.save(model, "best_model.pth")

            print("✅ Saved new best model and weights")

        if scheduler:
            scheduler.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_acc:.2f}%")


In [27]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50)

Epoch 1, Batch 0, Loss: 1.8997
Epoch 1, Batch 50, Loss: 1.5334
Epoch 1, Batch 100, Loss: 1.7539
Epoch 1, Batch 150, Loss: 1.5418
Epoch 1, Batch 200, Loss: 1.6964
Epoch 1, Batch 250, Loss: 1.4469
✅ Saved new best model and weights
Epoch [1/50], Loss: 1.6277, Val Loss: 1.5923, Val Accuracy: 38.42%
Epoch 2, Batch 0, Loss: 1.5281
Epoch 2, Batch 50, Loss: 1.5648
Epoch 2, Batch 100, Loss: 1.4906
Epoch 2, Batch 150, Loss: 1.4897
Epoch 2, Batch 200, Loss: 1.3886
Epoch 2, Batch 250, Loss: 1.3392
✅ Saved new best model and weights
Epoch [2/50], Loss: 1.3811, Val Loss: 1.2403, Val Accuracy: 58.20%
Epoch 3, Batch 0, Loss: 1.2498
Epoch 3, Batch 50, Loss: 1.2090
Epoch 3, Batch 100, Loss: 1.0186
Epoch 3, Batch 150, Loss: 1.1644
Epoch 3, Batch 200, Loss: 1.3617
Epoch 3, Batch 250, Loss: 0.8941
✅ Saved new best model and weights
Epoch [3/50], Loss: 1.1968, Val Loss: 1.1224, Val Accuracy: 61.13%
Epoch 4, Batch 0, Loss: 1.0025
Epoch 4, Batch 50, Loss: 0.9812
Epoch 4, Batch 100, Loss: 0.9815
Epoch 4, Batc

In [28]:
import numpy as np
import torch
from sklearn.metrics import f1_score, classification_report

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for batch in test_loader:
            # Handle various batch formats
            if isinstance(batch, tuple) and len(batch) == 2:
                images, labels = batch
            elif isinstance(batch, list) and len(batch) == 2:
                images, labels = batch[0], batch[1]
            else:
                raise ValueError(f"Unexpected DataLoader batch format: {type(batch)}, content: {batch}")

            # Convert string labels to integers if needed
            if isinstance(labels[0], str):
                label_mapping = {label: idx for idx, label in enumerate(set(labels))}
                labels = [label_mapping[label] for label in labels]

            # Convert to tensors
            if not isinstance(images, torch.Tensor):
                images = torch.tensor(np.array(images), dtype=torch.float32)
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(np.array(labels), dtype=torch.long)

            # Move data to the appropriate device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, preds = torch.max(outputs, 1)  # Get predicted class indices

            # Collect predictions and true labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate F1 score per emotion
    f1_scores = f1_score(all_labels, all_preds, average=None)
    overall_f1 = f1_score(all_labels, all_preds, average="weighted")
    report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(len(f1_scores))])

    print(f"F1 Scores per emotion: {f1_scores}")
    print(f"Overall Weighted F1 Score: {overall_f1}")
    print("\nClassification Report:\n")
    print(report)

    return report

In [29]:
rep = evaluate_model(model, test_loader, device)

F1 Scores per emotion: [0.80789061 0.87440683 0.55077173 0.66828676 0.5942492  0.80403458
 0.51792829]
Overall Weighted F1 Score: 0.7696187897617525

Classification Report:

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      2098
           1       0.85      0.90      0.87      1537
           2       0.63      0.49      0.55       695
           3       0.75      0.60      0.67       456
           4       0.66      0.54      0.59       172
           5       0.82      0.79      0.80       708
           6       0.63      0.44      0.52       148

    accuracy                           0.78      5814
   macro avg       0.73      0.66      0.69      5814
weighted avg       0.77      0.78      0.77      5814

